In [1]:
# Imports used in Selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from time import sleep

from bs4 import BeautifulSoup

# general python imports i.e. working with dataframes and basic maths
import pandas as pd
import numpy as np

import math
import datetime as datetime
from tqdm import tqdm_notebook as tqdm
import ast as ast

#dashboard imports
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table

# used to get most common words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, precision_score
pd.set_option('display.max_columns', None)

# Import some text packages
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

#folium doesn't seem like it works in dash, but really good code anyway
import folium
import os
import requests

import threading
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
% matplotlib inline


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Joey\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
UsageError: Line magic function `%` not found.


In [5]:
pwd

'C:\\Users\\Joey\\Google Drive\\5. Python Programs\\Webscraping\\2.Unfinished\\Chicken Dash'

# Entering Postcodes and getting restaurant URLs

In [22]:
#should cover all of greater london
postcode_list=['E8 1EX', 'E17 4PP', 'E6 6AB', 'SE7 8BL', 'SE2 0XT',
              'SW16 1BB', 'SW4 8EL', 'SW17 9NA', 'SW19 5AE', 'SW6 3PR', 
               'W5 5TH','NW8 9NH', 'NW6 1RZ','NW7 1NB','N8 8JD', 
               'E16 1XL', 'SE25 6AB', 'SE23 1NW', 'SE1 9SG','N21 1LE',
              'SE10 8JQ','EC1V 4NB','SW1V 2BP','W6 7NL','TW8 8JF','TW9 1DN',
              'BR1 3PX','N1 8AB']

In [27]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
#driver = webdriver.Chrome(executable_path='C:\\webdriver\\chromedriver.exe',chrome_options=options)

def shop_grabber2(postcode_list):
    driver = webdriver.Chrome(executable_path='C:\\Users\\Joey\\Desktop\\chromedriver.exe',chrome_options=options)
    
    for i in tqdm(postcode_list):
        
        driver.get('https://www.just-eat.co.uk/')
        
        post_field = driver.find_element_by_class_name("Form_c-search-input_2hlRP")
        
        post_field.clear()
        #post_field.click()
        post_field.send_keys(i)
        find_retaurants=driver.find_element_by_class_name("Form_c-search-btn_1EEhL")
        find_retaurants.click()

        

        for shop in driver.find_elements_by_class_name('c-listing-item-link'):
            url=shop.get_attribute('href')
            
            try:
                cusine=shop.find_element_by_class_name('c-listing-item-text').text
            except:
                cusine=None
                
            food_list2.append([url,cusine])

    
    return 

In [28]:
food_list2=[]


post1=postcode_list[0:14]
post2=postcode_list[14:28]
#post3=postcode_list[20:28]
# post4=postcode_list[21:28]


jobs = []


thread1 = threading.Thread(target=shop_grabber2, args=(post1,), )
thread2 = threading.Thread(target=shop_grabber2, args=(post2,), )
#thread3 = threading.Thread(target=shop_grabber2, args=(post3,), )
# thread4 = threading.Thread(target=shop_grabber2, args=(post4,), )
jobs.append(thread1)
jobs.append(thread2)
#jobs.append(thread3)
# jobs.append(thread4)

# Start the threads
for j in jobs:
    j.start()
    
# Ensure all of the threads have finished
for j in jobs:
    j.join()
    
len(food_list2)   

C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

use options instead of chrome_options



6958

In [29]:
url_df=pd.DataFrame(food_list2,columns=['url','cuisine']).drop_duplicates(['url'],keep='first')
url_df.to_csv('menu_urls.csv')

print('just eat restaurants in greater london=',len(url_df))

just eat restaurants in greater london= 3921


In [30]:
url_df=pd.read_csv('C:\\Users\\Joey\\Google Drive\\5. Python Programs\\Webscraping\\2.Unfinished\\Chicken Dash\\menu_urls.csv',index_col=0)

# Getting each restaurant's postcode, name and URL for review page - potential to scrape menu?

In [48]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

shops=[]

def shopgrabber3(restaurants):
    driver = webdriver.Chrome(executable_path='C:\\Users\\Joey\\Desktop\\chromedriver.exe',chrome_options=options)
    
    for i in tqdm(restaurants):
        
        driver.get(i)
        
        link_2_reviews=i.replace('/menu','/reviews')
        
        try:
            name=driver.find_element_by_xpath('//*[@id="restaurant"]/div[2]/div[1]/div[2]/h1').text
        except:
            try:
                name=driver.find_element_by_xpath('//*[@id="restaurant"]/div[2]/div[2]/div[2]/h1').text
            except:
                name=np.nan
                #print('bullshit with name')
                
        try:
            postcode=driver.find_element_by_xpath('//*[@id="postcode"]').text
        except:
            postcode=np.nan
            #print('bullshit with postcode')    
        
       
        
        shops.append([name,postcode,link_2_reviews,i])
        
    driver.quit()
        
    return

In [36]:
len(url_df)

3921

In [50]:
shops=[]

list1=url_df.url[0:1960]
list2=url_df.url[1866:3921]

jobs = []

thread1 = threading.Thread(target=shopgrabber3, args=(list1,), )
thread2 = threading.Thread(target=shopgrabber3, args=(list2,), )

jobs.append(thread1)
jobs.append(thread2)

# Start the threads
for j in jobs:
    j.start()
    
# Ensure all of the threads have finished
for j in jobs:
    j.join()
    
len(shops)   

C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning:

use options instead of chrome_options



4015

In [52]:
df=pd.DataFrame(shops,columns=['name','postcode','link_2_reviews','url']).merge(url_df,how='inner')
df['pc_prefix']=[str(x).split(' ')[0] for x in df.postcode]
df.to_csv('restaurants')

# Run From Here When Refreshing

In [3]:
df=pd.read_csv('restaurants',index_col=0)

# Scraping Ratings and Reviews

In [59]:
driver = webdriver.Chrome(executable_path='C:\\Users\\Joey\\Desktop\\chromedriver.exe')
driver.get('https://www.just-eat.co.uk/restaurants-telepizza-hornsey/reviews')

In [61]:
review_getter([['https://www.just-eat.co.uk/restaurants-telepizza-hornsey/reviews','Telepizza']])

C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning:

use options instead of chrome_options



In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

def review_getter(ziplist):    
    
    driver = webdriver.Chrome(executable_path='C:\\Users\\Joey\\Desktop\\chromedriver.exe'
                              ,chrome_options=options
                             )
    
    for i in tqdm(ziplist):
        
        
        try:
            driver.get(i[0])
            overall_rating=[]
            total_reviews=[]

            try:
                overall_rating=float(driver.find_element_by_xpath('//*[@id="restaurant"]/div[2]/div[2]/div[2]/span/span').text)
                overall_rating=round((overall_rating/6)*100,1)
                total_reviews=int(driver.find_element_by_xpath('//*[@id="restaurant"]/div[2]/div[1]/div[2]/div/p/a').text.split(' ')[0])
            except:
                overall_rating=np.nan
                total_reviews=np.nan


            dates=[]
            stars=[]
            reviews=[]

            #try:
            for review in driver.find_element_by_class_name('restaurantRatings').find_elements_by_css_selector('li'):

                #date
                try:
                    date=review.find_element_by_class_name('date').text
                    date2=datetime.datetime.strptime(date,"%d/%m/%Y").date()
                    dates.append(date2)
                except:
                    dates.append(np.nan)

                #rating
                try:
                    rating=float(review.find_elements_by_css_selector('img')[0].get_attribute('title').split(' ')[0])
                    rating2=round((rating/6)*100,1)
                    stars.append(float(rating2))
                except:
                    stars.append(np.nan)

                #review
                try:
                    #comment=review.text.split('\n')[2]
                    comment=review.find_element_by_class_name('comments').text
                    reviews.append(comment)
                except:
                    reviews.append(None)
    #         except:
    #             dates=np.nan
    #             stars=np.nan
    #             reviews=np.nan

            review_dict = {'overall_rating':overall_rating,
                           'total_reviews':total_reviews,
                            'date' : dates,
                           'stars' : stars,
                           'review_text' : reviews,
                           'link_2_reviews' : i[0],
                           'review_target': i[1]}


            #reviews_df = pd.DataFrame(reviews_list)

            review_df=pd.DataFrame(review_dict)

            all_reviews.append(review_df)
        except:
            pass
        
    driver.quit()
    
    return 

In [4]:
ziplist=list(zip(df.link_2_reviews,df.name))

In [5]:
all_reviews=[]

#Enter number of threads you want to try, and a list or df- to take the length of. and the name of your webscraping function..with no brackets or arguments
def jobber(thread_count,iteration_list,webscrape_func):

    jobs=[]
    
    #dividing the batch by number of threads
    x=int(round(len(iteration_list)/thread_count))
    remainder=int(((len(iteration_list)/thread_count)-x)*thread_count)
    
    for i in range(thread_count):
        #appending the first n-1 jobs to the job list.. if i=1 and x =25 then i'm appending list[0:25]
        jobs.append(threading.Thread(target=webscrape_func, 
                                     args=[iteration_list[(i-1)*x:(i*x)]]))
    
    #appending the last  job with remainder also added to the list
    jobs.append(threading.Thread(target=webscrape_func, 
                                 args=[iteration_list[((thread_count-1)*x):(thread_count*x)+remainder]]))

    
    # Start the threads
    for j in jobs:
        j.start()
    
    # Ensure all of the threads have finished
    for j in jobs:
        j.join()
    
    
    return 

jobber(5,ziplist,review_getter)

C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning:

use options instead of chrome_options



In [9]:
test=pd.concat(all_reviews).reset_index().drop(columns=['index'])
test.to_csv('reviews.csv')

In [4]:
#can apply NLP to test 
test=pd.read_csv('reviews.csv')
test['date_range']=test.date

test2=test.groupby('link_2_reviews').aggregate({'overall_rating':'max',
                                               'total_reviews':'max',
                                               'date':'max',
                                               #'date_range': lambda x: max(x) - min(x),
                                               'stars':'mean',
                                               'review_text':'count',
                                               'review_target':'first'}).reset_index()

df2=test2.merge(df)

In [5]:
df2=df2.rename(columns={'date':'most_recent_review','review_text':'reviews','stars':'trending_score'})
df2.trending_score=df2.trending_score.apply(lambda x: round(x,1))

In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3857 entries, 0 to 3856
Data columns (total 12 columns):
link_2_reviews        3857 non-null object
overall_rating        3838 non-null float64
total_reviews         3838 non-null float64
most_recent_review    3857 non-null object
trending_score        3857 non-null float64
reviews               3857 non-null int64
review_target         3819 non-null object
name                  3819 non-null object
postcode              3823 non-null object
url                   3857 non-null object
cuisine               3857 non-null object
pc_prefix             3823 non-null object
dtypes: float64(3), int64(1), object(8)
memory usage: 391.7+ KB


In [7]:
def to_date(string):
    y=int(string.split('-')[0])
    m=int(string.split('-')[1])
    d=int(string.split('-')[2])
    return datetime.date(y,m,d)

df2.most_recent_review=df2.most_recent_review.apply(to_date)

In [8]:
df3=df2[(df2.total_reviews>50) &
        (df2.most_recent_review>datetime.date(2019, 3, 1))]

len(df3)

2465

In [9]:
list_lists=[x.split(' ') for x in df2.cuisine]
flat_list = [item for sublist in list_lists for item in sublist]

from collections import Counter

Counter(flat_list)


Counter({'Burgers': 341,
         'Fast': 29,
         'Food': 33,
         'Halal': 1529,
         'African': 53,
         'Nigerian': 19,
         'Caribbean': 132,
         'English': 25,
         'Pizza': 782,
         'Italian': 448,
         'Gourmet': 30,
         'Breakfast': 68,
         'Grill': 188,
         'Korean': 12,
         'Asian': 44,
         'Kebab': 313,
         'Indian': 765,
         'Fusion': 10,
         'Sushi': 181,
         'Chinese': 478,
         'Chicken': 391,
         'Turkish': 141,
         'Milkshakes': 8,
         'American': 222,
         'Lebanese': 115,
         'Middle': 38,
         'Eastern': 43,
         'Curry': 438,
         'Thai': 181,
         'Oriental': 270,
         'South': 26,
         'Jamaican': 65,
         'Desserts': 98,
         'Dim': 13,
         'Sum': 13,
         'Seafood': 7,
         'Bangladeshi': 67,
         'Drinks': 15,
         'Peri': 242,
         'Fish': 112,
         '&': 112,
         'Chips': 112,
       

In [10]:
#Additional data prep

df3=df3.dropna()

chains=['kfc' , 'burger king' , 'subway' , 'pizza hut' , 'papa j']

df3['corporation']=['chain' if any(thing in x.lower() for thing in chains) else 'independent' for x in df3.name]
      
df3[df3.total_reviews>50].to_csv('test.csv')

postcodes=[]
for thing in df3.pc_prefix:
    if 'BR' in thing:
        a='BR'
    elif 'DA' in thing:
        a='DA'
    elif 'HA' in thing:
        a='HA'
    elif 'EN' in thing:
        a='EN'
    elif 'IG' in thing:
        a='IG'
    elif 'KT' in thing:
        a='KT'
    elif 'SM' in thing:
        a='SM'
    elif 'TW' in thing:
        a='TW' 
    elif 'UB' in thing:
        a='UB'
    elif 'WC1' in thing:
        a='WC1'    
    elif 'WC2' in thing:
        a='WC2'
    elif 'EC1' in thing:
        a='EC1'        
    elif 'EC2' in thing:
        a='EC2'        
    elif 'EC3' in thing:
        a='EC3'        
    elif 'EC4' in thing:
        a='EC4' 
    else:
        a=thing
    postcodes.append(a)
        

df3.pc_prefix=postcodes 

In [11]:
def latitude(x):
    try:
        a=requests.get('http://api.postcodes.io/postcodes/{}'.format(x)).json()['result']['latitude']
    except: 
        a=np.nan
    return a
        
def longitude(x):
    try:
        a=requests.get('http://api.postcodes.io/postcodes/{}'.format(x)).json()['result']['longitude']
    except: 
        a=np.nan
    return a

df3['latitude']=df3.postcode.apply(latitude)
df3['longitude']=df3.postcode.apply(latitude)



## Dash Code - Run in Anaconda Prompt for preview

In [ ]:
##using bootstrap

import numpy as np
import dash
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py
import base64

mapbox_access_token = 'pk.eyJ1Ijoiam9leTkxIiwiYSI6ImNqdDA3emFzazA5Y2IzeW56ZHRtN3Q3NG0ifQ.0l4idLnp8xRT8Y_Ipyj7vg'

#dataframe for maps and table
table_df = pd.read_csv('C:\\Users\\Joey\\Google Drive\\5. Python Programs\\Webscraping\\2.Unfinished\\Chicken Dash\\takeaway_data.csv',index_col=0)
table_df=table_df.rename(columns={'total_reviews':'Reviews ','trending_score':'Trending Score ','overall_rating':'Lifetime Score '})

table_df = table_df.sort_values('Trending Score ',ascending=False)
table_df=table_df.dropna()

zipped=zip(table_df.name
           ,table_df.postcode
           ,table_df['Lifetime Score '])

table_df['text']=[x +', '+y+', Rating: '+str(z) for x,y,z in zipped]

#for cuisine selector
acceptable=['Chicken','Pizza','Chinese','Italian','Caribbean','Burgers','Kebab','Sandwiches','British','Thai',
           'Vietnamese','Japanese','African','Indian','Curry','Mexican','Mediterranean']


#Human Theory Logo
image_filename = 'C:\\Users\\Joey\\Google Drive\\5. Python Programs\\Webscraping\\2.Unfinished\\Chicken Dash\\Human-Theory-Typemark-Sunset-Orange_RGB.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#top of site navigation bar

navbar = dbc.NavbarSimple(
    html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),height='70',
             width='140')
        ], href='https://www.humantheory.co.uk'),

#     brand="Human Theory",
#     brand_href="https://www.humantheory.co.uk/",
    sticky="top",
)


#2 containers in one row

body = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.H2("Best in your area"),
                        dcc.Dropdown(id='postSelector',
                                     options=[{'label': i, 'value': i} for i in sorted(table_df.pc_prefix.unique())],
                                     value='E8',
                                     style={'width': '90%'}),

#                        dcc.Dropdown(id='cuisineSelector2',
#                                     options=[{'label': i, 'value': i} for i in sorted(acceptable)],
#                                     value='Chicken',
#                                     style={'width': '90%'}),

                        html.Div(id='table-container')
                    ],
                    md=4,
                ),
                dbc.Col(
                    [
                       html.H2("Takeaway Map of London"),

                       dcc.Dropdown(id='cuisineSelector',
                                    options=[{'label': i, 'value': i} for i in sorted(acceptable)],
                                    value='Chicken',
                                    style={'width': '48%'}),

                       dcc.RadioItems(id='corpSelector',
                                      options=[{'label': i, 'value': i} for i in ['chain', 'independent']],
                                      value='independent',
                                      labelStyle={'display': 'inline-block'}),

                      dcc.Graph(id='area-shops')
                    ]
                ),
            ]
        )
    ],
    className="mt-4",
)

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN])

app.layout = html.Div([navbar, body])





#callbacks

def generate_table(dataframe):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
        html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(len(dataframe))]
    )

@app.callback(
    dash.dependencies.Output('table-container', 'children'),
    [dash.dependencies.Input('postSelector', 'value'),
     dash.dependencies.Input('cuisineSelector', 'value')])

def display_table(postSelector,cuisineSelector):

    #df = table_df[(table_df.pc_prefix==postSelector) ][['name',  'Trending Score ', 'Lifetime Score ','Reviews ']].head(10)

    df = table_df[(table_df.pc_prefix==postSelector) &
                  (table_df.cuisine.str.contains(cuisineSelector, regex=False))][['name',  'Trending Score ', 'Lifetime Score ','Reviews ']].head(10)

    return generate_table(df)



#map callback

@app.callback(
    dash.dependencies.Output('area-shops', 'figure'),
    [#dash.dependencies.Input('postSelector', 'value'),
    dash.dependencies.Input('cuisineSelector', 'value'),
    dash.dependencies.Input('corpSelector', 'value')
    ])

def update_graph(cuisineSelector='Chicken',corpSelector='independent'):


    df=table_df[#(table_df.pc_prefix==postSelector) &
                (table_df.cuisine.str.contains(cuisineSelector, regex=False))
                & (table_df.corporation==corpSelector)
    ]

    #red points
    data = [
        go.Scattermapbox(
            lat=df[df['Lifetime Score ']<=60]['latitude'],
            lon=df[df['Lifetime Score ']<=60]['longitude'],
            text = df[df['Lifetime Score ']<=60]['text'],
            mode='markers',
            marker=dict(size= 13,color = 'red',opacity = 1),
            name='shit food',
            showlegend=False
        ),

    #light red
        go.Scattermapbox(
            lat=df[(df['Lifetime Score ']>60) & (df['Lifetime Score ']<=70)]['latitude'],
            lon=df[(df['Lifetime Score ']>60) & (df['Lifetime Score ']<=70)]['longitude'],
            text = df[(df['Lifetime Score ']>60) & (df['Lifetime Score ']<=70)]['text'],
            mode='markers',
            marker=dict(size= 13,color = 'lightsalmon',opacity = 1),
            name='bad food',
            showlegend=False),


    #lawn green

        go.Scattermapbox(
            lat=df[(df['Lifetime Score ']>70) & (df['Lifetime Score ']<=80)]['latitude'],
            lon=df[(df['Lifetime Score ']>70) & (df['Lifetime Score ']<=80)]['longitude'],
            text = df[(df['Lifetime Score ']>70) & (df['Lifetime Score ']<=80)]['text'],
            mode='markers',
            marker=dict(size= 13,color = 'lawngreen',opacity = 1),
            name='good food',
            showlegend=False),

    #green

        go.Scattermapbox(
            lat=df[df['Lifetime Score ']>80]['latitude'],
            lon=df[df['Lifetime Score ']>80]['longitude'],
            text = df[df['Lifetime Score ']>80]['text'],
            mode='markers',
            marker=dict(size= 13,color = 'green',opacity = 1),
            name='excellent food',
            showlegend=False),
    ]

    layout = go.Layout(
        autosize=False,
        width=700,
        height=450,
        hovermode='closest',
        margin=go.layout.Margin(
        l=1,
        r=1,
         b=20,
         t=1,
#         pad=4
    ),
        mapbox=go.layout.Mapbox(accesstoken=mapbox_access_token,
                                bearing=0,
                                center=go.layout.mapbox.Center(lat=51.512324, lon=-0.075366),
                                pitch=0,
                                zoom=10),
    )

    return go.Figure(data=data, layout=layout)



if __name__ == '__main__':
    app.run_server(debug=True)